In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [6]:
# df = pd.read_excel('E:\ABD-work\Exact-names-sample10000.xlsx' , sheet_name=0 , header=None) # Semih hocanın gönderdiği dosyanın konumunu yazalım.
# df.rename({0:'OrcidID'} , axis = 1 , inplace=True)
# df.head()

,0,1,2,3,4,5,6,7,8
0,NaN,NPI,C0: Last Name,C0:First name,C0: Middle name,C1: Name to Display,affiliation,C16: email,OrcidID
1,0.0,1548237738,Merritt,Daniel,NaN,Daniel Merritt,Longview Regional Medical Center,dbmerrittjr@gmail.com,NaN
2,1.0,1326209347,Woods Md,Daniel,NaN,Daniel Woods,Orthopedics Specialists Of Northwest Indiana,daniel.woods@jeffersonhospital.org,NaN
3,2.0,1396743779,Adkison,David,NaN,David Adkison,Ascension St Vincents Birmingham,dadkison3009@charter.net,0000-0002-5789-8240
4,3.0,1639128002,Camarata,David,NaN,David Camarata,Arizona Bone And Joint Specialitst,david.camarata@honorhealth.com,NaN


In [ ]:
df['ScopusID'] = np.nan
# df['Scopus_MatchType'] = np.nan
# df['Scopus_NumOfResults'] = np.nan

In [ ]:
df['ScopusID'] = df['ScopusID'].astype('object')

In [ ]:
headers = {
    "User-Agent": "Buraya User-Agent bilgilerinizi yazacaksınız. Googleye My User Agent yazınca çıkıyor."
}

In [3]:
fuzzy_count = 0
exact_count = 0

In [ ]:
def get_exact_scopus_id(orcid):
    global fuzzy_count , exact_count
    try : 
        orcid = orcid
        r = requests.get(f"https://www.scopus.com/results/authorNamesList.uri?sort=count-f&src=al&orcidId={orcid}&sid=b7605a2c701a14c0d136648699739b87&sot=al&sdt=al&sl=36&s=AUTH--ORCID--ID%28{orcid}%29&selectionPageSearch=anl&reselectAuthor=false&activeFlag=true&showDocument=false&resultsPerPage=20&offset=1&jtp=false&currentPage=1&previousSelectionCount=0&tooManySelections=false&previousResultCount=0&authSubject=LFSC&authSubject=HLSC&authSubject=PHSC&authSubject=SOSC&exactAuthorSearch=false&showFullList=false&authorPreferredName=&origin=searchauthorfreelookup&affiliationId=&txGid=e4f338e3dab473a40424b29187e91bb0" , 
                 headers=headers)
        # exactAuthorSearch=false
        html = r.text
        soup = BeautifulSoup(html , 'html.parser')
        scopus_id = soup.find_all('input' , id = 'pageItems' , value = lambda x : x and x.isnumeric())[0]['value']
        exact_count += 1
        return scopus_id
    except : 
        fuzzy_count += 1

In [ ]:
for index, row in df.iterrows():
    orcid = row['OrcidID']
    
    scopus_id = None
    for i in range(2):
        scopus_id = get_exact_scopus_id(orcid)
        if scopus_id != None : 
            df.at[index, 'ScopusID'] = scopus_id
            break


In [ ]:
total = df.shape[0]
exact_rate = df[''] / total
fuzzy_rate = fuzzy_count / total

In [ ]:
print(f"""
      Total Count : {total}
      Exact Count : {exact_count} Exact Rate : {exact_rate}
      Fuzzy Count : {fuzzy_count} Fuzzy Rate : {fuzzy_rate} 
      """)

In [ ]:
# Şimdilik bu işimizi görür ama geliştirilecek kısımlar var